# Лабораторная работа №1 Мускатин КСм-21-1

Подключаем библиотеку findspark, для работы с большими данными с помощью технологии Spark

In [1]:
import findspark #для работы с большими данными

findspark.init()

Выполним подключение библиотеки pyspark и выполним запуск и инцилизацию сессии.  Назаовем документ сессии Lab_Muskatin

In [2]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Lab_Muskatin').getOrCreate() 

Загрузим данные из csv-файла. Первая строка является заголовками атрибтутов (header=true). Типизация автоматическая, Spark будет пытаться определить типы данных столбцов автоматически, основываясь на содержимом файла (inferSchema = True). 

In [3]:
df = spark.read.csv('EcommerceCustomers.csv', inferSchema = True, header = True)

Проверим корректность определения типа данных с помощью pyspark. 

In [4]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



Просмотрим наши исходные данные 

Можно утверждать что pyspark корректно определил типы данных.

In [5]:
df.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

Для создания вектора, содержащего характеристики, необходимо импортировать соответствующие библиотеки.

In [6]:
from pyspark.ml.linalg import Vectors

from pyspark.ml.feature import VectorAssembler

Давайте рассмотрим характеристики email, address и avatat. 

In [7]:
df.groupBy('Email').count().show()
df.groupBy('Address').count().show()
df.groupBy('Avatar').count().show()

+--------------------+-----+
|               Email|count|
+--------------------+-----+
|    dblair@gmail.com|    1|
|   yvaughn@meyer.com|    1|
|cbrady@gonzalez-m...|    1|
|russellbaldwin@fe...|    1|
|vancealicia@hotma...|    1|
|alvareznancy@luca...|    1|
|kcannon@alvarez-c...|    1|
|djohnson@hotmail.com|    1|
|mstephens@davidso...|    1|
|phillipscarrie@gm...|    1|
|christopher20@gma...|    1|
|  kelly24@lewis.info|    1|
| scottrose@yahoo.com|    1|
|  paulkemp@yahoo.com|    1|
| gharrison@davis.com|    1|
|   pvega@hotmail.com|    1|
| jstark@anderson.com|    1|
|    esmith@gmail.com|    1|
|alexandermichael@...|    1|
|rcarter@crane-tho...|    1|
+--------------------+-----+
only showing top 20 rows

+--------------------+-----+
|             Address|count|
+--------------------+-----+
|368 Morris Ports ...|    1|
|983 Ross GlenWest...|    1|
|036 Mary Drive Ap...|    1|
|186 Anthony Route...|    1|
|8594 Erica Manors...|    1|
|Unit 6538 Box 898...|    1|
|5086 Morales Isl

Можно сделать вывод, что атрибуты Email, Address и Avatar являются категориальными, так как они содержат большое количество уникальных значений, которые нельзя разбить на категории. Приведение их к числовому формату не имеет смысла, поэтому мы можем исключить эти атрибуты из анализа. 

Давайте выведем список всех атрибутов для удобства копирования.

In [8]:
df.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

Создадим вектор данных, указав атрибуты, которые будут использоваться на входе, и назовем выходные данные "features".

In [9]:
assembler = VectorAssembler(
    inputCols=[ 
        'Avg Session Length',
        'Time on App',
        'Time on Website',
        'Length of Membership'],
    outputCol = "features")

Преобразуем наши данные, используя новый векторный сборщик (assembler).

In [13]:
output = assembler.transform(df)
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



Теперь мы можем заметить, что появился новый атрибут "features", который содержит все характеристики. Давайте выведем первую строку данных, чтобы посмотреть на результат.

In [14]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

Наша целевая переменная - "Yearly Amount Spent". Теперь нам нужно сделать выборку из характеристик и целевой переменной. 

In [15]:
output.select("features","Yearly Amount Spent").show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

Выберем характеристики и целевую переменную из наших данных и запишем их в переменную "final_data".

In [16]:
final_data = output.select("features","Yearly Amount Spent")

Давайте разобьем наши данные на обучающую и тестовую выборки в соотношении 70/30. 70% данных будут использоваться для обучения, а 30% - для тестирования.

In [17]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

Импортируем библиотеку для выполнения регрессии и создадим экземпляр линейной регрессии.

In [18]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol = 'Yearly Amount Spent')

Выполним обучение нашей модели на данных в train_data

In [19]:
lrModel = lr.fit(train_data) 

Выполним вывод коэффициентов регрессии.

In [20]:
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.928625980246384,38.85623722110138,0.5673278797515255,61.43630076273912] Intercept: -1064.1273622170554


Давайте произведем оценку качества работы нашей модели на тестовых данных, используя метрику RMSE.

In [23]:
test_results = lrModel.evaluate(test_data)
print("RMSE: {}".format(test_results.rootMeanSquaredError))

RMSE: 9.703421245287053


Выведем описание данных 

In [24]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                363|
|   mean|  499.4642237588002|
| stddev|   76.5343877130778|
|    min|  282.4712457199145|
|    max|  725.5848140556806|
+-------+-------------------+



Видим, что ошибка соизмерима с данными целевой переменной

Выведем значение R²

In [25]:
print("R²: {}".format(test_results.r2))

R²: 0.9873370812936323


Наша модель объясняет ~99% данных.

# Корреляция Пирсона

Импортируем функцию corr, используемую для определения корреляции между атрибутами и целевой переменной.

In [26]:
from pyspark.sql.functions import corr

Давайте посмотрим на корреляцию между атрибутами Yearly Amount Spent и Avg Session Length.

In [27]:
df.select(corr('Yearly Amount Spent','Avg Session Length')).show()

+---------------------------------------------+
|corr(Yearly Amount Spent, Avg Session Length)|
+---------------------------------------------+
|                          0.35508829470764314|
+---------------------------------------------+



Значение коэффициента корреляции между атрибутами Yearly Amount Spent и Avg Session Length составляет около 0.355, что говорит о наличии умеренной прямой зависимости между ними.

Посмотрим на корреляцию между атрибутами Yearly Amount Spent и Time on App

In [28]:
df.select(corr('Yearly Amount Spent','Time on App')).show()

+--------------------------------------+
|corr(Yearly Amount Spent, Time on App)|
+--------------------------------------+
|                   0.49932777005345047|
+--------------------------------------+



Между атрибутами Yearly Amount Spent и Time on App наблюдается умеренная прямая зависимость, что подтверждается коэффициентом корреляции, равным ~0.5.

Рассмотрим корреляцию между атрибутами Yearly Amount Spent и Time on Website

In [29]:
df.select(corr('Yearly Amount Spent','Time on Website')).show()

+------------------------------------------+
|corr(Yearly Amount Spent, Time on Website)|
+------------------------------------------+
|                      -0.00264084467215...|
+------------------------------------------+



Коэффициент корреляции между Yearly Amount Spent и Time on Website близок к нулю (-0.00264084467215...), что означает практически отсутствие линейной зависимости между этими двумя атрибутами.

Посмотрим на корелляцию между Yearly Amount Spent и Length of Membership

In [30]:
df.select(corr('Yearly Amount Spent','Length of Membership')).show()

+-----------------------------------------------+
|corr(Yearly Amount Spent, Length of Membership)|
+-----------------------------------------------+
|                             0.8090835682475807|
+-----------------------------------------------+



Коэффициент корреляции между Yearly Amount Spent и Length of Membership равен примерно 0.809, что говорит о наличии сильной прямой зависимости между этими двумя атрибутами.

Проверим работу модели на данных, для которых не будут даны выводы.

In [33]:
unlabled_data = test_data.select('features')
unlabled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.7377203726281...|
|[30.8162006488763...|
|[30.8794843441274...|
|[30.9716756438877...|
|[31.1239743499119...|
|[31.1695067987115...|
|[31.2681042107507...|
|[31.2834474760581...|
|[31.3091926408918...|
|[31.4252268808548...|
|[31.4459724827577...|
|[31.5171218025062...|
|[31.6548096756927...|
|[31.6739155032749...|
|[31.7207699002873...|
|[31.8124825597242...|
|[31.8512531286083...|
|[31.8854062999117...|
|[31.9453957483445...|
+--------------------+
only showing top 20 rows



Посмотрим на прогнозы, которые мы сделали, относительно используемых атрибутов(features).

In [34]:
predictions = lrModel.transform(unlabled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...|331.17054776783743|
|[30.7377203726281...| 450.7092211904869|
|[30.8162006488763...|282.97665475889244|
|[30.8794843441274...| 493.7767834852948|
|[30.9716756438877...|  487.167839020771|
|[31.1239743499119...| 507.8517954734441|
|[31.1695067987115...| 417.3644306348574|
|[31.2681042107507...|426.88789301501697|
|[31.2834474760581...| 569.0821661921448|
|[31.3091926408918...|429.36722142394024|
|[31.4252268808548...| 534.5955017158647|
|[31.4459724827577...| 481.9957098408381|
|[31.5171218025062...|280.37489878245765|
|[31.6548096756927...| 468.7061976144778|
|[31.6739155032749...|  501.899084729607|
|[31.7207699002873...| 545.5362647502659|
|[31.8124825597242...|395.76189990150056|
|[31.8512531286083...|464.47211019534166|
|[31.8854062999117...| 398.9665880128962|
|[31.9453957483445...| 662.2727433297489|
+--------------------+------------